# Lidar Processing Using PDAL
This code processes raw lidar point clouds in order to calculate snow depth using PDAL. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy estimates. Therefore, little interpolation occurs under-canopy. We follow these protocols in order to obtain a 1-m rasterized product (as opposed to the 3-m rasterized product provided by ASO on the NSIDC data portal). NCALM and WSI flights were obtained through OpenTopography.

Start by importing necessary files

In [1]:
# import necessary files
import numpy as np 
import matplotlib.pyplot as plt
import json # where we will save the json files to run a pipeline
import os # for file management
import subprocess # allows us to run command line commands
import pdal # lidar processing package
import rasterio
import pyproj
from osgeo import gdal

import time
# packages to copy files
from pathlib import Path
import shutil
import glob
# packages to extract wkt from polygon
import shapefile
import pygeoif
# for parallel processing
import concurrent.futures
from multiprocessing import Pool
# see lidar_functions.py
import lidar_functions

Note that many functions are dependent on specific directory structures. See README

In [7]:
# Set current working directory
path = 'path/to/current/working/directory/'
os.chdir(path)
os.getcwd() # print to ensure we're in the right directory

'/Volumes/cpiske'

# Info
Get to know lidar file... <br>
In general, running PDAL python bindings can be difficult, so an easy workaround is to initiate a terminal command and run via subprocess. This is what we'll do for most of the processing. 

In [16]:
# define input file
input_lid = 'full/path/to/las/lidar.las'
pdal_info_cmd = ['pdal','info',input_lid] # general info
pdal_metadata_cmd = ['pdal','info',input_lid,'--metadata'] # full file metadata, including details crs
subprocess.run(pdal_metadata_cmd)

Save results to a dictionary <br>
This will be useful below when renaming files based on specific metadata

In [17]:
pdal_info_results = subprocess.run(pdal_metadata_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
pdal_info_dict = json.loads(pdal_info_results.stdout.decode())

# Retiling
Retiling lidar files can help with memory issues and allow for parallelization, speeding up processing. Standard tiles come in 500-1500m side lengths. Choose depending on point density and spatial extent of files. Many NCALM files come in 1000x1000m already. Buffer zones are dependent on point density etc. and allow for redundant classifications to avoid the edge effect.

Single Folder

In [ ]:
retile_input_path = 'path/to/input/lidar/folder/*.la*' # use * wildcard to identify all files that end in las or laz
retile_output_path = 'path/to/output/lidar/folder/#.laz' # # will be replaced by a number
# define retiling command and run 
retile_command = ['pdal', 'tile', retile_input_path,retile_output_path, '--length=1000','--buffer=50']
tic = time.perf_counter() # time process, ex. NCALM, 7 min
subprocess.run(retile_command)
toc = time.perf_counter()

Multiple subdirectories

In [ ]:
all_folders = [x[0] for x in os.walk('folder1/folder2/SCB/')]
# list indices of all folders that are called laz or las (i.e. 'folder1/folder2/SCB/flight1/laz/')
index_pos_list = [ i for i in range(len(all_folders)) if (all_folders[i][-3:] == 'laz' or all_folders[i][-3:] == 'las') ]
# create list of all folder paths that end with directory named 'laz' or 'las'
lid_list = [all_folders[i] for i in index_pos_list]

# for each of these folders...
for lid_folders in lid_list:
    # input wildcard string (ex. 'folder1/folder2/SCB/flight1/laz/*.laz')
    input_file_wildcard = lid_folders + '/*.'+ lid_folders[-3:] # [-3:] allows us to use .las or .laz
    # output pathname w/ wildcard (ex. 'folder1/folder2/SCB/flight1/retile/retile_#.laz')
    output_path = lid_folders[:-3] + '/retile/retile_#.' + lid_folders[-3:]
    # create pdal command
    retile_cmd = ['pdal', 'tile', input_file_wildcard, output_path,'--length=1000','--buffer=50']
    subprocess.run(retile_cmd)

## Rename
Many files come with inconsistent naming (including unsupported characters...) <br>
Rename all files to maintain consistency

In [ ]:
# rename a file with the lower left x and y defined as the corner point of the bounding box (add resolution/2 to get the center point of the box) 
# input - full lidar file path (i.e. 'lidar_files/filename.laz'), str
# See lidar_functions.py

def rename_llx_lly2(full_path):
    pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal command
    pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
    pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dict
    pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only pathname (i.e. 'lidar/lidar_files')
    name = os.path.join(pathname, str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))+full_path[-4:]) # create string with full pathname and llx and lly from dict
    os.rename(full_path, name) # rename file

In [ ]:
# rename a file with additional metadata at the beginning of the filename
# flight organization, watershed, date of flight (i.e. ASO_ICB_20140423)
# additional text will be taken from the flight directory name, could hardcode additional string instead of using folder name
# input - full lidar file path (i.e. 'lidar/lidar_files/filename.laz'), str
# See lidar_functions.py

def add_str_to_filename2(full_path):
    filename = os.path.basename(full_path) # isolate only filename (i.e 'filename.laz')
    pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only path name (i.e. 'lidar/lidar_files')
    add_str = os.path.normpath(pathname) # split up the path name (i.e. full path)
    add_str = [i for i in add_str.split(os.sep) if (i.startswith('ASO_') or i.startswith('NCALM_') or i.startswith('WSI_'))] # (i.e. 'ASO_SCB_2016')
    rename = os.path.join(pathname, add_str[0] + '_'+ filename) # add string to full pathname 
    os.rename(full_path, rename) # rename file

**Single file**

In [ ]:
# rename_llx_lly('filepath.laz')
# add_str_to_filename2('new_filepath.laz')

**Multiple Files**

In [ ]:
all_folders = [x[0] for x in os.walk('path/to/folder/')]
for folders in all_folders:
    # list indices of all folders that are called laz
    index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
    # save only those files 
    retile_list = [all_folders[i] for i in index_pos_list]
    for lidar_folder in retile_list:
        rename_retiles(lidar_folder)

**Parallel Processing**

In [ ]:

# save list of all subdirectories in specified path
all_folders = [x[0] for x in os.walk('folder1/folder2/folder3/')]
# list indices of all folders that are called 'retile', containing retiled lidar data
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# save only those files 
retile_list = [all_folders[i] for i in index_pos_list]
tic = time.perf_counter()
for lidar_folder in retile_list:
    if __name__ == '__main__':
        with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
            onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
            full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
            executor.map(lidar_function.rename_retiles, full_path) #running 10 times
toc = time.perf_counter()

In [ ]:
# time = 
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)
    executor.map(add_str_to_filename, full_path)
toc = time.perf_counter()

use if there are - in the output filename

In [78]:
# # use if there are - in the output filename
# pathname = 'Piske_lidar/MRB/Merced_lidar/ASO/ASO_MRB_20210429/tindex/tiles/'
# for files in [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]:
#     target = files.replace("-","")
#     target = pathname + target
#     full_path = pathname+files
#     os.rename(full_path,target)

# Save tile boundaries
Save sqlite (shp) of tile boundaries of retiles <br>
Alter filepaths 

In [ ]:
# save the tile index of a file to a new folder

# input - full lidar/sqlite file patsh (i.e. 'lidar/lidar_files/filename.laz'), [str, str]
# see lidar_functions.py
def create_tindex2(input_path, output_path):
    boundary_cmd = ['pdal', 'tindex', 'create', '--tindex', output_path, '--filespec', input_path, '-f', 'SQLite']
    subprocess.run(boundary_cmd)

**Parallel Processing**

In [ ]:
# one folder
tic = time.perf_counter()
lidar_folder = 'path/to/lidar/files/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
        full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
        # change depending on directory formats
        output_path = [os.path.join(os.path.basename(lidar_folder),'tindex' + s[:-3] + 'sqlite') for s in onlyfiles]
        executor.map(create_tindex, full_path, output_path) #running 10 times
toc = time.perf_counter()

In [10]:
# rename all retiled files
tic = time.perf_counter()
all_folders = [x[0] for x in os.walk('path/to/folder/')]
# list indices of all folders that are called laz
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# save only those files 
retile_list = [all_folders[i] for i in index_pos_list]
for lidar_folder in retile_list:
    if __name__ == '__main__':
        with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
            onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
            full_path = [lidar_folder + '/' + s for s in onlyfiles]
            output_path = [lidar_folder[:-6] + 'tindex/' + s[:-3] + 'sqlite' for s in onlyfiles]
            executor.map(create_tindex, full_path, output_path) #running 10 times
toc = time.perf_counter()

All files in ssd

In [ ]:
# tic = time.perf_counter()
# # list all folders in directory
# all_folders = [x[0] for x in os.walk('Piske_lidar/MRB/Merced_lidar/NCALM/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-3:] == 'laz' ]
# # save only those files 
# laz_list = [all_folders[i] for i in index_pos_list]
# for folders in laz_list:
#     pathname = folders + '/'
#     output_pathname = folders[:-3] + 'sqlite' + '/'
#     onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
#     for file in onlyfiles:
#         input_las = pathname + file
#         output_sqlite = output_pathname + file[:-4] + '.sqlite'
#         boundary_cmd = ['pdal', 'tindex', 'create', '--tindex', output_sqlite, '--filespec', input_las, '-f', 'SQLite']
#         subprocess.run(boundary_cmd)
# toc = time.perf_counter()

## Copy files
Save a copy of files that meet criteria into a new folder <br>
The goal is to decrease processing time for datum conversions in the MRB <br>
Change file extents depending on watershed

In [86]:
# copy file based on llx and lly into output filename. Note that this function relies on the assumption that files follow the structure 

# input_las 'folder1/folder2/folder3a/filename.laz' and output_las 'folder1/folder2/folder3b/filename.laz'
# if files are held in a different file structure, code must be changed to accomodate change
# input - full path to a las file
def copy_las_by_ext_ICB2(full_path):
    pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal command
    pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
    pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dict
    minx = round(pdal_info_dict['metadata']['minx'])
    miny = round(pdal_info_dict['metadata']['miny'])
    if (minx <= 288000 and minx >= 265000):
        if (miny >= 4165000 and miny<= 4180000):
            input_las = full_path
            output_las = os.path.join(os.path.dirname(os.path.realpath(os.path.dirname(os.path.realpath(full_path)))),'retile_ICB', os.path.basename(full_path))
            pdal_copy_cmd = ['pdal','translate', input_las, output_las]
            subprocess.run(pdal_copy_cmd)


**Parallel Processing**

In [38]:
tic = time.perf_counter()
all_folders = [x[0] for x in os.walk('path/to/lidar/flight/')]
# list indices of all folders that are called laz
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# save only those files 
retile_list = [all_folders[i] for i in index_pos_list]
for lidar_folder in retile_list:
    if __name__ == '__main__':
        with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
            onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
            full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
            executor.map(copy_las_by_ext_ICB, full_path) #running 10 times
toc = time.perf_counter()

copy tindex

In [68]:
# all_folders = [x[0] for x in os.walk('Piske_lidar/MRB/Merced_lidar/ASO')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-9:] == 'ICB_tiles' ]
# # save only those files 
# ICB_retile_list = [all_folders[i] for i in index_pos_list]
# for lidar_folder in ICB_retile_list:
#     onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#     for files in onlyfiles:
#         tindex_full_path = os.path.dirname(lidar_folder) + '/tindex/tiles/'+files[:-3]+'sqlite'
#         if os.path.isfile(tindex_full_path):
#             target_full_path = os.path.dirname(lidar_folder) + '/tindex/ICB_tiles/'
#             shutil.copy2(tindex_full_path, target_full_path)
    

# Datum Conversions

## Horizontal Datum Conversion

In [ ]:
horizontal_datum_conversion = 'lidar_processing/python_scripts/PDAL_workflow/JSON/horizontal_datum_conversion.json'

From PDAL library: https://pdal.io/tutorial/grid-shift.html

In [ ]:
# {
#     "pipeline": 
#         [
#             {
#                  "type": "readers.las",
#                  "filename": "#"
#             },
#             {
#                  "type": "filters.reprojection",
#                  "in_srs": "EPSG:2193",
#                  "out_srs": "EPSG:2193"
#             },
#             {
#                  "type": "writers.las",
#                  "filename": "#",
#                  "a_srs": "EPSG:2193", 
#                  "forward": "all"
#             }
#         ]
# }

In [ ]:
# input_las
# output_las = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272.las"

In [ ]:
# reader_dict = {"type":"readers.las",
#                "filename":input_las,
#               "spatialreference":"EPSG:26910"}

# reproject_dict = {"type":"filters.reprojection",
#         "in_srs":"EPSG:26910",
#         "out_srs":"EPSG:4326"}
# writer_dict = {"type":"writers.las",
#         "filename":output_las}

# pipeline_list = [reader_dict, reproject_dict, writer_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(test_datum_conversion, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)



In [ ]:
reader_dict = {"type":"readers.las"}

reproject_dict = {"type":"filters.reprojection","out_srs":"EPSG:26910"} #"in_srs":"EPSG:8999"
writer_dict = {"type":"writers.las",
               "a_srs":"EPSG:26910",
               "scale_x":0.00000001,
               "scale_y":0.00000001,
              "offset_x":"auto",
              "offset_y":"auto"}

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(horizontal_datum_conversion, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
reader_dict = {"type":"readers.las",
              "filename":input_reproj_hor}

reproject_dict = {"type":"filters.reprojection",
                  "in_srs":"+proj=utm +zone=10 +datum=WGS84 +units=m +no_defs",
                  "out_srs":"EPSG:26910"} #"in_srs":"EPSG:8999"
writer_dict = {"type":"writers.las",
               "a_srs":"EPSG:26910",
              #  "scale_x":0.00000001,
              #  "scale_y":0.00000001,
              # "offset_x":"auto",
              # "offset_y":"auto",
              "filename":output_reproj_hor}

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(horizontal_datum_conversion, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
input_reproj_hor = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAVD88/mcc_part_b_tile_000_002.laz'
output_reproj_hor = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAVD88/test1/apr_test_hor.laz'

In [ ]:
# Single Files
reader = '--readers.las.filename='+input_reproj_hor
writer = '--writers.gdal.filename='+output_reproj_hor
horizontal_conversion_cmd = ['pdal', 'pipeline', horizontal_datum_conversion]
subprocess.run(horizontal_conversion_cmd)

In [ ]:
# converts one file
pdal_commands = ['pdal', 'translate', input_reproj_hor, output_reproj_hor, '--json', horizontal_datum_conversion]
subprocess.run(pdal_commands)

In [ ]:
# # converts one file
# pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', test_datum_conversion]
# subprocess.run(pdal_commands)

In [ ]:
# # converts all the files in a folder
# pathname = 'path_to_folder'
# output_pathname = "path_to_folder"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', horizontal_datum_conversion]
#     subprocess.run(pdal_commands)

In [ ]:
# convert datums of all three ASO flights
ASO_paths = glob.glob("lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/retile/ASO*/", recursive = True) #create list with only the ASO retiled folders
for paths in ASO_paths: # for each flight
    # name output using indices of the date in the filename
    output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83/"+ paths[-9:-1]
    onlyfiles = [f for f in os.listdir(paths) if os.path.isfile(os.path.join(paths, f))] #create list with only the ASO retiled files
    for file in onlyfiles:    # for each file within the date's retiled output
        input_las = paths + file # the input las file is the ASO flight path + the tile's filename
        output_las = output_pathname + file # the output las is the same filename
        pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', horizontal_datum_conversion]
        subprocess.run(pdal_commands)

## Vertical Datum Conversion
using output of code above

In [62]:
vertical_datum_json = "cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/vertical_datum_conversion.json"

In [ ]:
reader_dict = {"type":"readers.las",
              "filename":input_las_reproj}

reproject_dict = {"type":"filters.reprojection",
                  "in_srs": "EPSG:26910",
                  "out_srs":"+init=EPSG:26910 +geoidgrids=/Volumes/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/g2012bu5.gtx +t_epoch=2014.441"}
                  #"out_srs":"+init=epsg:26910 +geoidgrids=/Volumes/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/g2012bu5.gtx"} # change 6339 to 26910

writer_dict = {"type":"writers.las",
               "a_srs":"EPSG:26910+5703",
              "filename":output_las_reproj}
#"a_srs":"EPSG:5703"}  slightly shifted horizontally?5498

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(vertical_datum_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
# converts single file
input_las_reproj = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAVD88/test1/apr_test_hor.laz'
output_las_reproj = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAVD88/test1/apr_test_vert.laz'

In [ ]:
vertical_datum_conversion = ['pdal', 'pipeline', vertical_datum_json]
subprocess.run(vertical_datum_conversion)

In [ ]:
# # converts all the files in a folder
# pathname = 'path_to_folder'
# output_pathname = "path_to_folder"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', vertical_datum_json]
#     subprocess.run(pdal_commands)

In [ ]:
# tic = time.perf_counter()
# # convert datums of all three ASO flights - will get warnings about deprecated crs
# ASO_paths_vert = glob.glob("lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83/*/", recursive = True) #create list with only the ASO retiled folders
# for paths in ASO_paths: # for each flight
#     # name output using indices of the date in the filename
#     output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAVD88/"+ paths[-9:]
#     onlyfiles = [f for f in os.listdir(paths) if os.path.isfile(os.path.join(paths, f))] #create list with only the ASO retiled files
#     for file in onlyfiles:    # for each file within the date's retiled output
#         input_las = paths + file # the input las file is the ASO flight path + the tile's filename
#         output_las = output_pathname + file # the output las is the same filename
#         pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', vertical_datum_json]
#         subprocess.run(pdal_commands)
# toc = time.perf_counter() # 51 min

SCB NCALM 2014 from NAD83/NAVD88 to NAD83/NAD83

In [68]:
input_las_reproj = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/original/NCALM_SCB_2014_734000_4365000.laz'
output_las_reproj = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/test2/NCALM_SCB_2014_734000_4365000.laz'

In [69]:
reader_dict = {"type":"readers.las",
              "filename":input_las_reproj}

reproject_dict = {"type":"filters.reprojection",
                  "in_srs": "+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs",
                  "out_srs":"+proj=utm +zone=10 +datum=NAD83 +units=m +vunits=m +no_defs"}
                  #"out_srs":"+init=epsg:26910 +geoidgrids=/Volumes/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/g2012bu5.gtx"} # change 6339 to 26910

writer_dict = {"type":"writers.las",
               "a_srs":"+proj=utm +zone=10 +datum=NAD83 +units=m +vunits=m +no_defs",
              "filename":output_las_reproj}
#"a_srs":"EPSG:5703"}  slightly shifted horizontally?5498

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(vertical_datum_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [70]:
vertical_datum_conversion = ['pdal', 'pipeline', vertical_datum_json]
subprocess.run(vertical_datum_conversion)

CompletedProcess(args=['pdal', 'pipeline', 'cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/vertical_datum_conversion.json'], returncode=0)

In [43]:
# # best option for now 
# input_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/original/NCALM_SCB_2014_734000_4365000.laz'
# output_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/test13/NCALM_SCB_2014_734000_4365000_vert.laz'
# reproj_fil_in_srs = '--filters.reprojection.in_srs=EPSG:26910+5703'
# reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26910 +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx +t_epoch=2010.0'
# writers_compression = '--writers.las.compression=true'
# writers_a_srs = '--writers.las.a_srs=+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs' 

In [113]:
# best option for now 
input_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/ICB/2021/ASO_MRB_20210429_637354_453052.laz'
output_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/ICB/2021/ASO_MRB_20210429_637354_453052_reproj2.laz'
reproj_fil_in_srs = '--filters.reprojection.in_srs=EPSG:102003'
reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:32611'# +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx +t_epoch=2010.0'
writers_compression = '--writers.las.compression=true'
writers_a_srs = '--writers.las.a_srs=EPSG:32611' 

In [218]:
# best option for now 
input_las = r'Macintosh HD - Data/Users/carapiske/Desktop/TH_las_2013.laz'
output_las = r'Macintosh HD - Data/Users/carapiske/Desktop/TH_las_2013_reproj.laz'
reproj_fil_in_srs = '--filters.reprojection.in_srs=EPSG:4978'
reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26913 +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx'# +t_epoch=2010.0'
writers_compression = '--writers.las.compression=true'
writers_a_srs = '--writers.las.a_srs=EPSG:26913' 

In [219]:
vert_datum_reproj = ['pdal','translate','-i', input_las, '-o', output_las, 'reprojection', reproj_fil_in_srs, reproj_fil_out_srs, writers_compression, writers_a_srs]
subprocess.run(vert_datum_reproj)

(pdal translate filters.reprojection Error) GDAL failure (1) File not found or invalid


CompletedProcess(args=['pdal', 'translate', '-i', 'Macintosh HD - Data/Users/carapiske/Desktop/TH_las_2013.laz', '-o', 'Macintosh HD - Data/Users/carapiske/Desktop/TH_las_2013_reproj.laz', 'reprojection', '--filters.reprojection.in_srs=EPSG:4978', '--filters.reprojection.out_srs=+init=EPSG:26913 +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx', '--writers.las.compression=true', '--writers.las.a_srs=EPSG:26913'], returncode=0)

In [50]:
input_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83NAD83/20160417/original/ASO_SCB_20160417_734530_4365782.laz'
output_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83NAD83/20160417/test3/ASO_SCB_20160417_734530_4365782_hor.laz'
reproj_fil_in_srs = '--filters.reprojection.in_srs='
reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26910'# +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx'# +t_epoch=2010.0'
#writers_compression = '--writers.las.compression=true'
writers_a_srs = '--writers.las.a_srs=+proj=utm +zone=10 +datum=NAD83 +units=m +no_defs' 

In [42]:
input_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83NAD83/20160417/original/ASO_SCB_20160417_734530_4365782.laz'
output_las = 'cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/ASO_to_NAD83NAD83/20160417/test3/ASO_SCB_20160417_734530_4365782.laz'
reproj_fil_in_srs = '--filters.reprojection.in_srs=+proj=utm +zone=10 +datum=WGS84 +units=m +no_defs'
reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26910 +geoidgrids=/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx'# +t_epoch=2010.0'
writers_compression = '--writers.las.compression=true'
writers_a_srs = '--writers.las.a_srs=+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs' 

# Noise Filter

In [185]:
outlier_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/outlier_filter.json'
filter_outlier = {"type":"filters.outlier",
                  "method":"statistical",
                  "mean_k":8,
                  "multiplier":2.2}
filter_range = {"type":"filters.range",
                "limits":"Z[1700:2700]"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(outlier_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [186]:
input_las = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/NCALM_SCB_2014_730000_4365000.las"
output_las = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/outlier_filter/NCALM_SCB_2014_730000_4365000.las"
outlier_cmd = ['pdal', 'translate',input_las, output_las, '--json', outlier_json]
subprocess.run(outlier_cmd)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/NCALM_SCB_2014_730000_4365000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/outlier_filter/NCALM_SCB_2014_730000_4365000.las', '--json', '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/outlier_filter.json'], returncode=0)

# Ground Filter
Modified from https://pdal.io/tutorial/ground-filters.html <br>
Bradley Chambers

In [ ]:
# name JSON file
gf_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json'

In [ ]:
# name JSON file
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json'

# create a pipeline and save to a json file 
#last_val = True
filter_dict_reproj = {'type':'filters.reprojection', #PDAL’s default parameters are specified in meters, and individual filter stages typically assume that units are at least uniform in X, Y, and Z. Because data will not always be provided in this way, PDAL pipelines should account for any data reprojections and parameter scaling that are required from one dataset to the next
                      'in_srs':'EPSG:26910',
                      'out_srs': "EPSG:26910",}

filter_dict_assign = {'type':'filters.assign', # set the value of a dimension for all points to a provided value that pass a range filter
                      "assignment":"Classification[:]=0"} #  single option has been provided that specifies the dimension, range, and value to assign. In this case, we are stating that we would like to apply a value of 0 to the Classification dimension for every point

filter_dict_elm = {'type':'filters.elm'} # identify low noise points that can adversely affect ground segmentation algorithms, automatically assigns value of 7 

filter_dict_outlier = {'type':'filters.outlier'} #  two methods of outlier detection at the moment: radius and statistical. Both aim to identify points that are isolated and likely arise from noise sources, classify values as 7
# classify ground points
filter_dict_smrf = {"type":"filters.smrf",
                    "ignore":"Classification[7:7]"}#, # ignore outliers
                    #"slope":0.3,
                    #"window":16,
                    #"threshold":0.15,
                    #"scalar":1.2}

filter_dict_range = {"type":"filters.range",
                    "limits":"Classification[2:2]"}
#output_las = "lidar_processing/python_scripts/PDAL/test_las/ncalm_2014_732000_4373000_GFtutorial.las"


pipeline_list = [filter_dict_reproj,filter_dict_assign,filter_dict_elm, filter_dict_outlier, filter_dict_smrf, filter_dict_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(gf_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
pdal_commands = ['pdal', 'translate', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4363000.laz', "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/test/ot_736000_4363000.laz", '--json', output_json]
subprocess.run(pdal_commands)

In [ ]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/retile/NCALM_2014/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', gf_json]
    subprocess.run(pdal_commands)

In [ ]:
#gf_tutorial_command = ['pdal', 'translate', input_las, output_las, '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json']

In [ ]:
#subprocess.run(gf_tutorial_command)

In [ ]:
# for i in [0, 1, 2, 3, 4, 5]:
#     input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/NAD83_NAVD88/20160518/mcc_part_b_tile_004_00'+str(i) + '.las'
#     output_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.las'
#     gf_tutorial_command = ['pdal', 'translate', input_las, output_las, '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json']
#     subprocess.run(gf_tutorial_command)

### Pre-classified ground filter

In [59]:
range_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_preClassified.json'
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [62]:
input_las = '/Volumes/Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.las'
output_las = '/Volumes/Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip_ground.las'

In [63]:
range_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
subprocess.run(range_cmd)

CompletedProcess(args=['pdal', 'translate', '/Volumes/Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.las', '/Volumes/Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip_ground.las', '--json', '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_preClassified.json'], returncode=0)

In [216]:
def ground_filter(input_las, output_las):
    range_cmd = ['pdal', 'translate', input_las,  output_las, '--json','/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_preClassified.json']
    subprocess.run(range_cmd)

In [7]:
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]

In [9]:
#329.1668573822826s
tic = time.perf_counter()
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered'
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + '/' + s for s in onlyfiles]
        output_path_full = [output_path + '/' + s for s in onlyfiles]
        executor.map(ground_filter, full_path, output_path_full) #running 10 times
toc = time.perf_counter()

### Last Returns

In [152]:
returns_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_by_last_return.json'
filter_range = {"type":"filters.returns",
                "groups":"last,only"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(returns_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [153]:
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/last_returns/'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
for files in onlyfiles:
    full_input_path = input_path + '/' + files
    full_output_path = output_path + files[:-4]+'_#'+'.las'
    range_cmd = ['pdal', 'translate', full_input_path,  full_output_path, '--json',returns_json]
    subprocess.run(range_cmd)

## HAG

Replace Z values with HAG instead of elevation

In [214]:
HAG_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/height_above_ground_DEM.json'

In [254]:
# convert all z values to the height above ground 
target_dem = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM_ground/NCALM_SCB_2014_BE.tif'
filter_hag = {"type":"filters.hag_dem",
              "raster":target_dem,
              "zero_ground":"false"}
filter_ferry = {"type":"filters.ferry",
                "dimensions":"HeightAboveGround=>Z"}
filter_range = {"type":"filters.range",
                "limits":"Z[-0.2:70]"}
pipeline_list = [filter_hag, filter_ferry,filter_range]#,filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(HAG_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [222]:
input_las = "Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/NAD83_NAD83_epoch2010/ASO_SCB_20160326_730777_4364591.laz"
output_las = "Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/ASO_SCB_20160326_730777_4364591.laz"
HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
subprocess.run(HAG_cmd)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/NAD83_NAD83_epoch2010/ASO_SCB_20160326_730777_4364591.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/ASO_SCB_20160326_730777_4364591.laz', '--json', '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/height_above_ground_DEM.json'], returncode=0)

In [221]:
input_las = "Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/NAD83_NAD83_epoch2010/ASO_SCB_20160326_730777_4365426.laz"
output_las = "Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/ASO_SCB_20160326_730777_4365426.laz"
HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
subprocess.run(HAG_cmd)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/NAD83_NAD83_epoch2010/ASO_SCB_20160326_730777_4365426.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/ASO_SCB_20160326_730777_4365426.laz', '--json', '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/height_above_ground_DEM.json'], returncode=0)

In [257]:
# multiple files
pathname = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/"
output_pathname = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
    subprocess.run(HAG_cmd)

In [256]:
# # multiple files
# pathname = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/laz/"
# output_pathname = "Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
#     subprocess.run(HAG_cmd)

In [160]:
# parallel processing
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/laz/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + '/' + s for s in onlyfiles]
        output_path_full = [output_path + '/' + s for s in onlyfiles]
        executor.map(HAG_dem, full_path, output_path_full) #running 10 times
toc = time.perf_counter()

In [226]:
# parallel processing
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/NAD83_NAD83_epoch2010/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + '/' + s for s in onlyfiles]
        output_path_full = [output_path + '/' + s for s in onlyfiles]
        executor.map(HAG_dem, full_path, output_path_full) #running 10 times
toc = time.perf_counter()

In [227]:
# parallel processing
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/NAD83_NAD83_epoch2010/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + '/' + s for s in onlyfiles]
        output_path_full = [output_path + '/' + s for s in onlyfiles]
        executor.map(HAG_dem, full_path, output_path_full) #running 10 times
toc = time.perf_counter()

Add a new dimension

In [ ]:
# this works to add a new dimension
output_las_hag = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/vertical_bias/rasterize/ASO_20160417_hag_newDim.laz'
hag_addDim = ['pdal', 'translate', input_las_hag, output_las_hag, 'hag_dem', '--filters.hag_dem.raster=SCB/NCALM_2014_bulk_be/NCALM_2014_be/east_HD1_241.TIF', '--writers.las.extra_dims=HeightAboveGround=float32']
subprocess.run(hag_addDim)

# Clip to Polygon

## Move files
Make a copy of road covered tiles. <br>
The spatial extents of the road polygon are 738382,4371452 : 738686,4372063 <br>
chose only files with a min x > 737000 and a min y > 4370000

In [258]:
# defining source and destination
# paths
src = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/*.la*'
trg = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

In [231]:
# defining source and destination
# paths
src = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/*.la*'
trg = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/hwy89_vertical_bias/target_las'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

## Merge
Merge files over the road

In [259]:
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/*.la*' # define path of input files
output_fname = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_hwy89_merge.laz'# set output filename
input_fname = glob.glob(input_path) # save to list
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

CompletedProcess(args=['pdal', 'merge', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_738000_4371000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_738000_4372000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_738000_4373000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_739000_4371000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_739000_4372000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_740000_4371000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_740000_4372000.las', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_740000_4373000.las', 'Piske_l

In [233]:
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/*.la*' # define path of input files
output_fname = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_hwy89_merge.las'# set output filename
input_fname = glob.glob(input_path) # save to list
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

CompletedProcess(args=['pdal', 'merge', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_737530_4370782.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_737530_4371782.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_738530_4370782.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_738530_4371782.laz', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_las/ASO_SCB_20160417_hwy89_merge.las'], returncode=0)

In [234]:
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/*.la*' # define path of input files
output_fname = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las'# set output filename
input_fname = glob.glob(input_path) # save to list
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

CompletedProcess(args=['pdal', 'merge', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_737399_4370936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_737399_4371936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_738399_4370936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_738399_4371936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las'], returncode=0)

In [246]:
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/*.la*' # define path of input files
output_fname = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las'# set output filename
input_fname = glob.glob(input_path) # save to list
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

CompletedProcess(args=['pdal', 'merge', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_737399_4370936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_737399_4371936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_738399_4370936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_738399_4371936.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las'], returncode=0)

## Clip

In [127]:
# name JSON file
clip_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/clip_to_geometries.json'

In [236]:
# extract wkt from hwy 89 polygon
hwy89 = shapefile.Reader("/Volumes/Piske_lidar/SCB/supporting_files/bounding_box/hwy89_poly.shp")
geom=[]
for s in hwy89.shapes():
    geom.append(pygeoif.geometry.as_shape(s)) 
poly_base = pygeoif.MultiPolygon(geom)

In [54]:
# create a pipeline and save to a json file 
filter_crop = {'type':'filters.crop',
                 'polygon':poly_base.wkt}
pipeline_list = [filter_crop]
pipeline_dict = {'pipeline' : pipeline_list}
with open(clip_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [261]:
input_las = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_hwy89_merge.laz'
output_las = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.laz'

In [252]:
input_las = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_las/ASO_SCB_20160518_hwy89_merge.las'
output_las = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.las'

In [262]:
pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', clip_json]
subprocess.run(pdal_commands)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_las/NCALM_SCB_2014_hwy89_merge.laz', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.laz', '--json', '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/clip_to_geometries.json'], returncode=0)

Parallel Processing

## Calculate lowest 10th percentile

### Convert file to text

In [134]:
writer_text_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/writer_text.json'

In [ ]:
reader_las = {"type":"readers.las",
              "filename":input_las_txt}
writer_text = {"type":"writers.text",
               "format":"csv",
               "order":"Z",
               "keep_unspecified":"false",
               "filename":output_las_txt}
pipeline_list = [reader_las, writer_text]
pipeline_dict = {'pipeline' : pipeline_list}
with open(writer_text_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
# either use this with files specified in the pipeline
txt_cmd = ['pdal','pipeline',writer_text_json]
subprocess.run(txt_cmd)

We'll use this workflow for now

In [263]:
# or use this with no pipeline
input_las = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.laz'
output_txt = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.csv'
txt_cmd = ['pdal', 'translate', input_las, output_txt, '-w', 'writers.text', '--writers.text.format=csv','--writers.text.order=Z','--writers.text.keep_unspecified=false']
subprocess.run(txt_cmd)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.laz', 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.csv', '-w', 'writers.text', '--writers.text.format=csv', '--writers.text.order=Z', '--writers.text.keep_unspecified=false'], returncode=0)

In [266]:
# or use this with no pipeline
input_las = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.las'
output_txt = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.csv'
txt_cmd = ['pdal', 'translate', input_las, output_txt, '-w', 'writers.text', '--writers.text.format=csv','--writers.text.order=Z','--writers.text.keep_unspecified=false']
subprocess.run(txt_cmd)

CompletedProcess(args=['pdal', 'translate', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.las', 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.csv', '-w', 'writers.text', '--writers.text.format=csv', '--writers.text.order=Z', '--writers.text.keep_unspecified=false'], returncode=0)

### read text and calculate lowest 10th percentile

In [270]:
# calculate stats
output_las_txt = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.csv'
#output_las_txt = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.csv'
hag_arr = np.loadtxt(output_las_txt,skiprows=1)
lowest_10th_per = np.nanpercentile(hag_arr, 10)
mean_hag = np.nanmean(hag_arr)
median_hag = np.nanmedian(hag_arr)
# NCALM_SCB_2014_hwy89stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
# ASO_SCB_20160326_hwy89stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
# ASO_SCB_20160417_hwy89stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
# ASO_SCB_20160518_hwy89stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]

In [271]:
print(NCALM_SCB_2014_hwy89stats)
print(ASO_SCB_20160326_hwy89stats)
print(ASO_SCB_20160417_hwy89stats)
print(ASO_SCB_20160518_hwy89stats)

['lowest_10th', -0.08, 'mean', 0.011861657597321053, 'median', 0.0]
['lowest_10th', 0.21, 'mean', 0.29112474437627817, 'median', 0.29]
['lowest_10th', 0.0, 'mean', 0.0, 'median', 0.0]
['lowest_10th', 0.0, 'mean', 0.0, 'median', 0.0]


In [ ]:
# # input - path to csv file
# def calculate_vertical_bias(input_text_file):
#     hag_arr = np.loadtxt(output_las_txt,skiprows=1)
#     lowest_10th_per = np.nanpercentile(hag_arr, 10)
#     mean_hag = np.nanmean(hag_arr)
#     median_hag = np.nanmedian(hag_arr)
#     stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
#     return(stats)

In [ ]:
ASO_20160326_hwy89_stats = calculate_vertical_bias(output_las_txt)
ASO_20160417_hwy89_stats = calculate_vertical_bias(output_las_txt)
ASO_20160518_hwy89_stats = calculate_vertical_bias(output_las_txt)

In [ ]:
NCALM_2014_hwy89_stats = calculate_vertical_bias(output_las_txt)
NCALM_2014_hwy89_stats

In [ ]:
print(ASO_20160326_hwy89_stats)
print(ASO_20160417_hwy89_stats)
print(ASO_20160518_hwy89_stats)

In [ ]:
# input_las - HAG, las file clipped to the road
# output_path - path to output files
# base_name - string, typically flight name
def calculate_vertical_bias(input_las, output_path, base_name):
    # convert height only to txt file
    txt_cmd = ['pdal', 'translate', input_las, output_las_txt, '-w', 'writers.text', '--writers.text.format=csv','--writers.text.order=Z','--writers.text.keep_unspecified=false']
    subprocess.run(txt_cmd)
    # calculate stats
    hag_arr = np.loadtxt(output_las_txt,skiprows=1)
    lowest_10th_per = np.nanpercentile(hag_arr, 10)
    mean_hag = np.nanmean(hag_arr)
    median_hag = np.nanmedian(hag_arr)
    stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
    return(stats)

In [ ]:
ASO_20160326_hwy89_stats = calculate_vertical_bias('lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/clipped/ASO_20160326/ASO_20160326_clipped.laz', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/vertical_bias/ASO_20160326/', 'ASO_2060326')
ASO_20160417_hwy89_stats = calculate_vertical_bias('lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/clipped/ASO_20160417/ASO_20160417_clipped.laz', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/vertical_bias/ASO_20160417/', 'ASO_2060417')
ASO_20160518_hwy89_stats = calculate_vertical_bias('lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/clipped/ASO_20160518/ASO_20160518_clipped.las', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/vertical_bias/ASO_20160518/', 'ASO_2060518')
NCALM_2014_hwy89_stats = calculate_vertical_bias('lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/clipped/NCALM_2014/NCALM_2014_clipped.laz', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/vertical_bias/NCALM_2014/', 'NCALM_2014')


In [ ]:
print(ASO_20160326_hwy89_stats)
print(ASO_20160417_hwy89_stats)
print(ASO_20160518_hwy89_stats)
print(NCALM_2014_hwy89_stats)

## Correct ASO files

In [273]:
assign_json = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/assign.json'
value_assign = 'Z = Z - ' + str(.21)
filter_assign = {"type":"filters.assign",
                 "value":value_assign}
pipeline_list = [filter_assign]
pipeline_dict = {'pipeline' : pipeline_list}
with open(assign_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [283]:
# multiple files
pathname = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/HAG/'
output_pathname = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_las/'
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', assign_json]
    subprocess.run(HAG_cmd)

# Vegetation Classifications

## Filter By Veg Strata
Note that there are no ground points above 2m

In [ ]:
range_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_neg0pt15_0pt15.json'
filter_range = {"type":"filters.range",
                "limits":"Z[-0.15:0.15)"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
tic = time.perf_counter()
pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/laz/"
output_pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes_originalNCALM/nonground/laz/'
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
    subprocess.run(strata_cmd)
toc = time.perf_counter()

In [275]:
filter_range_2_nonground = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_2_nonground.json'
filter_range_2_ground = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_2_ground.json'
filter_range_2 = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_2.json'
filter_range_neg0pt15_0pt15 = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_neg0pt15_0pt15.json'
filter_range_0pt15_2 = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/filter_range_0pt15_2.json'

In [278]:
# 374
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_pathname = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_0pt15_2/laz/'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
for file in onlyfiles:
    input_las = input_path + file
    output_las = output_pathname + file
    strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',filter_range_0pt15_2]
    subprocess.run(strata_cmd)
toc = time.perf_counter()

PDAL: readers.las: Invalid VLR - exceeds specified file range.



In [ ]:
tic = time.perf_counter()
pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes_originalNCALM/HAG/"
output_pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes_originalNCALM/veg_strata_2_nonground/'
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',filter_range_2_nonground]
    subprocess.run(strata_cmd)
toc = time.perf_counter()

In [ ]:
toc-tic

In [ ]:
# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes/HAG/"
# output_pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes/veg_strata_neg0pt15_0pt15/'
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     strata_neg0pt15_0pt15_cmd = ['pdal', 'translate', '-i', input_las, '-o', output_las, '-f', 'range', '--filters.range.limits="Z[-0.15:0.15)"']
#     subprocess.run(strata_neg0pt15_0pt15_cmd)

# Rasterize

In [3]:
rasterize_json_mean = 'cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif_mean.json'
rasterize_json_count = 'cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif_count.json'


In [286]:
# name JSON file
#rasterize_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif_count.json'
# input_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000.las' # define input las full file path
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000_GFtutorial.las' # define input las full file path

# create a pipeline and save to a json file 
reader_dict = {'type':'readers.las'}
filter_gdal= {"type": "writers.gdal",
              'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7'}#,
             #'window_size':4}
#output_dtm = "lidar_processing/python_scripts/PDAL/test_file/ncalm_2014_732000_4373000_DTMtutorial.tif"


pipeline_list = [reader_dict,filter_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(rasterize_json_mean, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [196]:
# Single Files
reader = '--readers.las.filename='+'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered/NCALM_SCB_2014_730000_4365000.las'
writer = '--writers.gdal.filename='+'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM_ground/NCALM_SCB_2014_730000_4365000_4.tif'
rasterize_command = ['pdal', 'pipeline', rasterize_json_mean, writer, reader]
subprocess.run(rasterize_command)

CompletedProcess(args=['pdal', 'pipeline', 'cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif_mean.json', '--writers.gdal.filename=Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM_ground/NCALM_SCB_2014_730000_4365000_4.tif', '--readers.las.filename=Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered/NCALM_SCB_2014_730000_4365000.las'], returncode=0)

parallel processing files from one folder (worth it)

In [4]:
# 179.46s
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_neg0pt15_0pt15/laz/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_neg0pt15_0pt15/tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + '/' + s for s in onlyfiles]
        full_output_tif = [output_path + '/' + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(rasterize_count, full_input_las, full_output_tif) 
toc = time.perf_counter()

PDAL: Unable to write GDAL data with no points for output.

PDAL: Unable to write GDAL data with no points for output.

PDAL: Unable to write GDAL data with no points for output.



In [5]:
# 179.46s
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_0pt15_2/laz/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_0pt15_2/tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + '/' + s for s in onlyfiles]
        full_output_tif = [output_path + '/' + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(rasterize_count, full_input_las, full_output_tif) 
toc = time.perf_counter()

PDAL: Unable to write GDAL data with no points for output.

PDAL: Unable to write GDAL data with no points for output.

PDAL: Unable to write GDAL data with no points for output.



In [7]:
# 179.46s
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_gt2_nonground/laz/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_gt2_nonground/tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + '/' + s for s in onlyfiles]
        full_output_tif = [output_path + '/' + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(rasterize_count, full_input_las, full_output_tif) 
toc = time.perf_counter()

In [284]:
# 179.46s
tic = time.perf_counter()
input_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_las/'
output_path = 'Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + '/' + s for s in onlyfiles]
        full_output_tif = [output_path + '/' + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(rasterize_mean, full_input_las, full_output_tif) 
toc = time.perf_counter()

single files

In [52]:
# # Single Files
# writer = '--writers.gdal.filename='+output_raster_filepath
# reader = '--readers.las.filename='+input_las_filepath
# rasterize_command = ['pdal', 'pipeline', rasterize_json_mean, writer, reader]
# subprocess.run(rasterize_command)

CompletedProcess(args=['pdal', 'pipeline', 'cpiske/lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif_mean.json', '--writers.gdal.filename=cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/NCALM_SCB_2014_734000_4365000_ground.tif', '--readers.las.filename=cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toNAD83NAD83/2014/NCALM_SCB_2014_734000_4365000_ground.las'], returncode=0)

multiple files

In [145]:
# # multiple files
# tic = time.perf_counter()
# pathname = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered/'
# output_pathname = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM_ground/'
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     writer = '--writers.gdal.filename='+output_pathname+file[:-4]+".tif"
#     reader = '--readers.las.filename='+pathname+file
#     rasterize_command = ['pdal', 'pipeline', rasterize_json_mean, writer, reader]
#     subprocess.run(rasterize_command)
# toc = time.perf_counter()

multiple folders

In [ ]:
# tic = time.perf_counter()
# # list all folders in directory
# all_folders = [x[0] for x in os.walk('lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/veg_classes_originalNCALM/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-3:] == 'laz' ]
# # save only those files 
# laz_list = [all_folders[i] for i in index_pos_list]
# for folders in laz_list:
#     pathname = folders + '/'
#     output_pathname = folders[:-3] + 'tif' + '/'
#     onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
#     for file in onlyfiles:
#         input_las = pathname + file
#         writer = '--writers.gdal.filename='+output_pathname+file[:-4]+".tif"
#         reader = '--readers.las.filename='+pathname+file
#         rasterize_command = ['pdal', 'pipeline', rasterize_json_count, writer, reader]
#         subprocess.run(rasterize_command)
# toc = time.perf_counter()

test script

In [ ]:
# # name JSON file
# rasterize_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json'
# # input_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000.las' # define input las full file path
# # output_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000_GFtutorial.las' # define input las full file path

# # create a pipeline and save to a json file 
# reader_dict = {'type':'readers.las'}
# filter_gdal= {"type": "writers.gdal",
#               'gdaldriver':'GTiff',
#               'output_type': 'mean',
#               'resolution': 1.0}
# #output_dtm = "lidar_processing/python_scripts/PDAL/test_file/ncalm_2014_732000_4373000_DTMtutorial.tif"


# pipeline_list = [reader_dict,filter_gdal]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(rasterize_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

In [ ]:
# See below for better solution

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--writers.gdal.resolution=1','--writers.gdal.output_type=mean']
#     subprocess.run(pdal_commands)

In [ ]:
# didn't work because the pipeline wants an input/output

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', rasterize_json]
#     subprocess.run(pdal_commands)

In [ ]:
# test new formatting
# input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# output_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]

In [ ]:
# test on ASO data
# for i in [0, 1, 2, 3, 4, 5]:
#     writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
#     reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.las'
#     rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]
#     subprocess.run(rasterize_command)

### Clipping Geometries
The goal of this portion of the code is to clip the raster based on the control areas (in the case of SCB, hwy 89)<br>
See: https://pdal.io/tutorial/clipping/index.html#clipping

In [ ]:
[
    "autzen.laz",
    {
      "type":"filters.overlay",
      "dimension":"Classification",
      "datasource":"attributes.vrt",
      "layer":"OGRGeoJSON",
      "column":"CLS"
    },
    {
      "type":"filters.range",
      "limits":"Classification[5:5]"
    },
    "output.las"
]

In [ ]:
# name JSON file
clipping_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/clip_las_to_shp.json'

filter_overlay_dict = {"type":"filters.overlay",
                       "dimension":"Classification",
                       "datasource":"SCB/bounding_box/hwy89_poly.shp",
                       "column":"OBJECTID"}
filter_range_dict = {"type":"filters.range",
                     "limits":"Classification[4193:4193]"}

pipeline_list = [filter_overlay_dict,filter_range_dict]
with open(clipping_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/reproj_NAD83_NAVD88/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
pdal_clip = ['pdal', 'translate', input_las, output_las, '--json', output_json]
subprocess.run(pdal_clip)

# File format conversions

## Convert .las to .txt
see: https://pdal.io/stages/writers.text.html

In [ ]:
# set up json file commands
output_txt = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000Test.asc'
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_txt.json'

# create a pipeline and save to a json file 

filter_dict = {'type':'readers.las',
               'override_srs': "EPSG:4326",
              'filename': input_las} # we are reading in a las file
rasterize_dict = {'type':'writers.las',
'format':'geojson',
'order':'X,Y,Z',
'keep_unspecified':'false',
'filename':output_txt}


pipeline_list = [filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
json_path = 'lidar_processing/python_scripts/PDAL/JSON/las_to_txt.json'
pdal_commands = ['pdal', 'pipeline', json_path]
subprocess.run(pdal_commands)

### .laz to .las

In [ ]:
# # set up json file commands
# input_laz = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.laz'
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.las'
# output_json = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'

# # create a pipeline and save to a json file 

# filter_dict = {'type':'readers.las',
#                'filename': input_las} # we are reading in a las file
# translate_dict = {'type':'writers.las',
#                   "a_srs": "EPSG:4326",
#                   'filename':output_las}


# pipeline_list = [filter_dict, translate_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(output_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
# json_path = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'
# pdal_commands = ['pdal', 'pipeline', json_path]
# subprocess.run(pdal_commands)

# Raster Caluclations

In [ ]:
for i in [0,1,2,3,4,5]:
    apr_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_00'+str(i)+'.tif'
    may_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
    output = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/raster_subtract/mcc_part_b_tile_004_00'+str(i)+'.tif'
    raster_sub = ['gdal_calc.py', '-a', apr_elev, '-b', may_elev, '--calc="a - b"', '--outfile', output]
    subprocess.run(raster_sub)

# Theo Code

In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

z_min = 0.15
z_max = 2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.tif'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_tif.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'GTiff',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

# z_min = 0.15
# z_max = 2
# z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.asc'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_asc.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'XYZ',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz

where path_to_laz_folder is the path to the LAS/LAZ file (you just need the folder path, not the file path).

:/input is the new folder that will be created in your Docker container that will hold your point cloud.

0b is just the image id of pdal

/input/test.laz is the path to the point cloud in the Docker container.


In [ ]:
path_to_laz_folder = 'lidar_processing/python_scripts/PDAL/test_las'


In [ ]:
sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz


In [ ]:
import json
import subprocess
import pdal

def assemblePipeline(input_las, list_of_dicts):
    pipeline_list = [input_las]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline' : pipeline_list}
    return pipeline_dict

def makeHeightFilter(height, buffer):
    z_min = height - buffer/2
    z_max = height + buffer/2
    z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
    heightDict = {'type':'filters.range', 'limits':z_range}
    return heightDict

def makeRasterizeFilter(output_raster, resolution, epsg):
    rasterize_dict = {'filename':output_raster,
                      'gdaldriver':'GTiff',
                      'output_type':'count',
                      'resolution':resolution,
                      'override_srs' : epsg,
                      'type': 'writers.gdal'}
    return rasterize_dict

def convertTifForPIL(input_raster, output_raster, epsg):
    ''' GDAL bindings are an alien concept to me, so I gave up and used
        subprocess.'''
    commands = ['gdal_translate', input_raster, output_raster, '-ot', 'Byte', '-a_srs', epsg]
    subprocess.run(commands)


def buildHeightSlice(input_las, height, buffer, output_raster, resolution, epsg, json_path=None):
    filter_dict = makeHeightFilter(height, buffer)
    rasterize_dict = makeRasterizeFilter(output_raster, resolution, epsg)
    filter_list = [filter_dict, rasterize_dict]
    pipeline_dict = assemblePipeline(input_las, filter_list)

    if json_path is not None:
        with open(json_path, 'w') as out:
            json.dump(pipeline_dict, out, indent=4)
        pdal_commands = ['pdal', 'pipeline', json_path]
        subprocess.run(pdal_commands)
    else:
        pdal_commands = json.dumps(pipeline_dict)
        pipeline = pdal.Pipeline(pdal_commands)
        pipeline.execute()

input_las = '/Users/theo/data/las/TLS_0244_20180612_01_v003_30m_clip_height_norm.las'
height = 1.37
buffer = 0.05
z_min = height - buffer/2
z_max = height + buffer/2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
temp_raster = '/Users/theo/Pictures/almost_cool.tif'
final_raster = '/Users/theo/Pictures/cool.tif'
resolution = 0.01
epsg = 'EPSG:3310'

buildHeightSlice(input_las, height, buffer, temp_raster, resolution, epsg)
convertTifForPIL(temp_raster, final_raster, epsg)

In [ ]:
import json
import subprocess
import pdal
import argparse

# Create flags for the user to utilize.
parser = argparse.ArgumentParser(description="Generate JSON pipeline to generate DTM from a point cloud.")
      
required = parser.add_argument_group('Required arguments')
required.add_argument('-crs', '--coordinate_system', required=True, action='store', help="EPSG code.")
required.add_argument('-i', '--infile', required=True, action='store', help="Input path to point cloud")
required.add_argument('-o', '--outfile', required=True, action='store', help="Output path.")
args = parser.parse_args()

def generateJSON(infile, list_of_dicts):
    pipeline_list = [infile]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline': pipeline_list}
    with open("pipeline.json", 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

def generateDTM(epsg, infile, outfile):
    reproject_dict = {"type": "filters.reprojection",
                      "out_srs": "EPSG:{}".format(epsg)}
    reclassify_zero_dict = {"type": "filters.assign",
                       "assignment": "Classification[:]=0"}
    elm_dict = {"type": "filters.elm"}
    outlier_dict = {"type": "filters.outlier"}
    smrf_dict = {"type": "filters.smrf", "ignore": "Classification[7:7]",
                 "slope": 0.2, "window": 16, "threshold": 0.45, "scalar": 1.2}
    range_dict = {"type":"filters.range", "limits":"Classification[2:2]"}
    output_dict = {"filename": outfile, "gdaldriver": "GTiff", "output_type": "all", "resolution": 0.01, "type": "writers.gdal"}
    list_of_dicts = list([reproject_dict, reclassify_zero_dict, elm_dict, outlier_dict, smrf_dict, range_dict, output_dict])
    generateJSON(infile, list_of_dicts)
    pdal_cmds = ['pdal', 'pipeline', 'pipeline.json']
    subprocess.run(pdal_cmds)
    
generateDTM(args.coordinate_system, args.infile, args.outfile)

# More helpful things

### Get stats of a dataset
see: https://www.spatialised.net/lidar-qa-with-pdal-part-1/

In [ ]:
# name JSON file

stats_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/stats.json'

In [ ]:
reader_las = {"type":"readers.las",
              "filename": input_las_stats}
filter_stats = {"type":"filters.stats",
                "dimensions":"Z",
                "global":"Z",
                "advanced":"true"}
pipeline_list = [reader_las, filter_stats]
#pipeline_dict = {reader_las, filter_stats}
# with open(stats_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
pipeline_list

In [ ]:
pipeline = pdal.Pipeline(json.dumps(pipeline_list))
pipeline.execute()

In [ ]:
json.loads(pipeline.metadata)["metadata"]["filters.stats"]["statistic"]

## Navigating folders/files

**create list of files/folders with a wildcard (*)**

In [40]:
# ex. list all files in folder4 that end in .laz = folder1/folder2/folder3/folder4/*.laz
# ex. list all folders named folder3 in folder 1 = foler1/*/folder3
# ex. list all list all contents in folder2 = folder2/* - note just folder 2, no subdirectories
glob_cmd = 'path'
glob_exe = glob.glob(file_glob_cmd)

**create a list with all directories/subdirectories on a path**

In [ ]:
all_folders = [x[0] for x in os.walk('path')]

**create a list with all directories/subdirectories on a path with specific folders**

In [52]:
all_folders = [x[0] for x in os.walk('path')]
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ] # must change this to meet requirements
full_list = [all_folders[i] for i in index_pos_list]

**get name of the directory just above one listed**

In [38]:
subdirname = os.path.basename(os.path.dirname('path'))

**create a list with only filenames**

In [ ]:
onlyfiles = [f for f in os.listdir('path') if os.path.isfile(os.path.join('path', f))]

**create a list with full file paths**

In [47]:
full_path = ['path' + '/' + s for s in onlyfiles]

## Parallel Processing

In [ ]:
# time = 142.5
tic = time.perf_counter()
list(map(function, args));
toc = time.perf_counter()

In [ ]:
# time = 53 s
tic = time.perf_counter()
if __name__ == "__main__":
    pool = Pool(3)
    pool.map(function, arg)
    pool.close()
toc = time.perf_counter()

In [ ]:
# time = very fast? .06s
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()

In [ ]:
# time = 0.22
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ProcessPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()